In [43]:
import pandas as pd
import numpy as np
import re
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from rapidfuzz.fuzz import WRatio
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
options = Options()
options.add_argument("--headless") 


from IPython.core.display import HTML



In [36]:


targeted_element_example = """
<a 
    href="https://www.linkedin.com/company/google" 
    rel="noopener" 
    target="_self" 
    data-testid="result-title-a" 
    data-handled-by-react="true" 
    class="eVNpHGjtxRBq_gLOfGDr LQNqh2U1kzYxREs65IJu"

    <span>Google - LinkedIn</span>
</a>
"""

#print(targeted_element)
#print(targeted_element)

In [37]:
# mock_data_df = pd.read_csv(r"C:\Users\jcrit\New Coding Project Folder\mock_data.csv")
mock_data = ['Redhat','Google','Microsoft','Taco Bell']
# mock_data
# #mock_data_df

In [38]:
def linkedIn_search(company) -> list:
    try:
        query = f'{company} site:linkedin.com'
        #take a company
        #perform a search
        #query one time and quit?
        #go to duckduckgo.com
        driver.get('https://duckduckgo.com')

        #print(f'driver title:{driver.title}')
        #once at the site,look for the duckduckgo.com class result-title-a
        #data-testid has value result-title-a
        #have to send the keys to the search bar 

        search_bar = driver.find_element(By.NAME,'q')
        search_bar.clear()
        search_bar.send_keys(query)
        search_bar.send_keys(Keys.RETURN)

        time.sleep(random.uniform(1,2))
        #print(f'driver title is now:{driver.title}')

        links = WebDriverWait(driver,10).until(
            EC.presence_of_all_elements_located((By.XPATH,'(//a[@data-testid="result-title-a"])[position() <= 2]'))
        ) 
        #print(f'found the following anchor tags:{links}')
        res = []

        #within the loop, we need to ensure the link matches the desired structure
        #use a regex function
        #pattern: linkedin.com/company/companyname
        pattern = re.compile(r'.*linkedin\.com/company/[^/]+/?')

        for link in links:
            href = link.get_attribute('href')
            #print(f'current url is:{href}')
            if href and pattern.match(href):
                res.append(href)
        
        return res
    except Exception as e:
        print(f"Error during search for {company}: {e}")
        return []

In [39]:
#dictionary of 'company:linkedin.com/company' URL is the output of the function below
#take a list of company names, find the top two company LinkedIn URLs, and return a dictionary of company:LinkedIn URLs
def company_linked_dict(companies:list) -> dict:

    result = {}

    #loop through the list of companies. apply the search to each one
    #apply linkedIn_search function

    for company in companies:
        urls = linkedIn_search(company)
        #account for sometimes there not being at least two URLs returned
        result[company] = urls + [None] * (2 - len(urls))
    #print(result)
    return result

In [40]:
#This function will: 
    #1. Convert the dictionary into a dataframe
    #2. Compare the two linkedIn links (specifically the 'company name' inside the link) against the original company name
    #3. Return a dataframe with two columns:
        #Column 1: original company name 
        #Column 2: closest matching LinkedIn URL

def compare_and_return_best_result(dictionary: dict) -> pd.DataFrame:
    # Convert the dictionary into a DataFrame
    df = pd.DataFrame.from_dict(
        dictionary, orient='index', columns=['LI Match1', 'LI Match2']
    )
    
    # Extract company names from LinkedIn URLs
    df['LI Name1'] = df['LI Match1'].apply(extract_company_name)
    df['LI Name2'] = df['LI Match2'].apply(extract_company_name)

    # Calculate similarity scores
    df['Match 1 Similarity %'] = [
        WRatio(a, b) if a and b else 0  # Ensure non-null values
        for a, b in zip(df.index, df['LI Name1'])
    ]

    df['Match 2 Similarity %'] = [
        WRatio(a, b) if a and b else 0
        for a, b in zip(df.index, df['LI Name2'])
    ]

    # Determine the closest match
    df['Closest Match'] = np.where(
        # If Match 1 is higher and above 65%
        (df['Match 1 Similarity %'] >= df['Match 2 Similarity %']) & (df['Match 1 Similarity %'] > 65),
        df['LI Match1'],
        # Otherwise, check if Match 2 is above 65%, else NaN
        np.where(
            (df['Match 2 Similarity %'] > 65),
            df['LI Match2'],
            np.nan
        )
    )

    # Display the DataFrame for debugging (optional)
    #display(df)

    # Return the resulting DataFrame
    df = df[['Closest Match']]
    df = df.reset_index().rename(columns = {'index':'Company'})
    return df


#also need a function for extracting the company name from the URL


def extract_company_name(url) -> str:
    if url is None:
        return None
    #takes a url
    #perform regex to pull out the company name
    match = re.search(r'.*linkedin\.com/company/([^/]+)/?$',url)
    return match.group(1) if match else None

    

In [ ]:
driver = webdriver.Chrome(options = options)

#replace mock_data with whatever list of company names you want
resulting_df = compare_and_return_best_result(company_linked_dict(mock_data))
display(resulting_df)


driver.quit()

